In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# preliminaries: load relevant libraries; import data; define colour palette
#---------------------------------------------------------------------------------------------------------------------------
# load libraries
import brewer2mpl
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
import re
import numpy as np
import nltk
import tensorflow as tf
import random
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch
from tensorflow.keras.layers import LSTM, SpatialDropout1D

# set seeds for reproducibility purposes
np.random.seed(5)
tf.random.set_seed(5)
random.seed(5)
os.environ['PYTHONHASHSEED'] = str(5)

# import data 
data = pd.read_csv("sona.csv")

# Generate the RdGy colour palette
# num_colors = 11
# rdgy_palette = brewer2mpl.get_map('RdGy', 'Sequential', num_colors, reverse=True).mpl_colors

#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# data pre-processing: prepare data  ~ subsettting by presidents, cleaning, and segmenting speeches into sentences
#---------------------------------------------------------------------------------------------------------------------------

import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import re

# Assuming `data` is your original DataFrame
# Select subset of four out of six presidents
subset = data[data['president'].isin(['Mandela', 'Mbeki', 'Zuma', 'Ramaphosa'])]

# Initialize a list to store the sentences
sentences_data = []

# Iterate through each row in the subset
for index, row in subset.iterrows():
    # Split the speech into sentences
    speech_sentences = sent_tokenize(row['speech'])
    
    # For each sentence, create a new row with the same information
    for sentence in speech_sentences:
        sentences_data.append({
            'sentence': sentence,
            'year': row['year'],
            'president': row['president'],
            'date': row['date']
        })

# Create a new DataFrame with sentences
sona_sentences = pd.DataFrame(sentences_data)

# Filtering function to remove stop words and only words with a length of three characters or more
english_words = set(nltk.corpus.words.words())
stop_words = set(stopwords.words('english'))

def filter_text(text):
    return ' '.join([word for word in text.split() if word not in stop_words and len(word) > 3 and word in english_words])

# Apply the filter function to the cleaned sentences
sona_sentences['cleaned_sentence'] = sona_sentences['sentence'].apply(filter_text)

# Clean sentences
sona_sentences['cleaned_sentence'] = sona_sentences['cleaned_sentence'].apply(lambda text: re.sub(r'[^A-Za-z\s]', '', text).lower())

# Output the number of rows
sona_sentences.shape

#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
sona_sentences.head()

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# data pre-processing: create three different data structures for analysis
#---------------------------------------------------------------------------------------------------------------------------
# create BoW ~ using top 150 words
bow_vectorizer = CountVectorizer(max_features=150)
bow_features = bow_vectorizer.fit_transform(sona_sentences['cleaned_sentence']).toarray()
bow_features.shape
# create tf-idf ~ using 150 words 
tfidf_vectorizer = TfidfVectorizer(max_features=150)
tfidf_features = tfidf_vectorizer.fit_transform(sona_sentences['cleaned_sentence']).toarray()

# create embeddings
tokenized_speeches = [text.split() for text in sona_sentences['cleaned_sentence']]
model = Word2Vec(sentences=tokenized_speeches, vector_size=150, window=5, min_count=1, workers=4)
embeddings_features = np.array([np.mean([model.wv[word] for word in text.split() if word in model.wv] or [np.zeros(150)], axis=0) for text in sona_sentences['cleaned_sentence']])
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# create splits for the data ~ 60-20-20 = training-validation-test
#---------------------------------------------------------------------------------------------------------------------------
seed = 5

# create data split for BoW approach
labels = sona_sentences['president']
X_train_bow, X_temp_bow, y_train, y_temp = train_test_split(bow_features, labels, test_size=0.4, random_state=seed, stratify=labels)
X_val_bow, X_test_bow, y_val, y_test = train_test_split(X_temp_bow, y_temp, test_size=0.5, random_state=seed, stratify=y_temp)

# create data split for tf-idf approach
X_train_tfidf, X_temp_tfidf = train_test_split(tfidf_features, test_size=0.4, random_state=seed, stratify=labels)
X_val_tfidf, X_test_tfidf = train_test_split(X_temp_tfidf, test_size=0.5, random_state=seed, stratify=y_temp)

# create data split for embedding approach
X_train_emb, X_temp_emb = train_test_split(embeddings_features, test_size=0.4, random_state=seed, stratify=labels)
X_val_emb, X_test_emb = train_test_split(X_temp_emb, test_size=0.5, random_state=seed, stratify=y_temp)


#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Classification tree  ~ for BoW approach
#---------------------------------------------------------------------------------------------------------------------------
tree_params = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]}
# hyperparameter gridsearch 
tree_clf_bow = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=5) 
tree_clf_bow.fit(X_train_bow, y_train)

# predictions and evaluation
y_pred_tree_bow = tree_clf_bow.predict(X_test_bow)
print("Best parameters:", tree_clf_bow.best_params_)
print("\nClassification Report (Classification Tree - BoW):")
print(classification_report(y_test, y_pred_tree_bow))

# confusion matrix
conf_matrix_tree_bow = confusion_matrix(y_test, y_pred_tree_bow)
sns.heatmap(conf_matrix_tree_bow, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Classification tree  ~ for tf-idf approach
#---------------------------------------------------------------------------------------------------------------------------
tree_params = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]}
# hyperparameter gridsearch 
tree_clf_tfidf = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=5) 
tree_clf_tfidf.fit(X_train_tfidf, y_train)

# predictions and evaluation
y_pred_tree_tfidf = tree_clf_tfidf.predict(X_test_tfidf)
print("Best parameters:", tree_clf_tfidf.best_params_)
print("\nClassification Report (Classification Tree - tf-idf):")
print(classification_report(y_test, y_pred_tree_tfidf))

# confusion matrix
conf_matrix_tree_tfidf = confusion_matrix(y_test, y_pred_tree_tfidf)
sns.heatmap(conf_matrix_tree_tfidf, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Classification tree  ~ for embedding approach
#---------------------------------------------------------------------------------------------------------------------------
tree_params = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]}
# hyperparameter gridsearch 
tree_clf_emb = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=5) 
tree_clf_emb.fit(X_train_emb, y_train)

# predictions and evaluation
y_pred_tree_emb = tree_clf_emb.predict(X_test_emb)
print("Best parameters:", tree_clf_emb.best_params_)
print("\nClassification Report (Classification Tree - embedding):")
print(classification_report(y_test, y_pred_tree_tfidf))

# confusion matrix
conf_matrix_tree_emb = confusion_matrix(y_test, y_pred_tree_emb)
sns.heatmap(conf_matrix_tree_emb, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Feed-forward neural network ~ for BoW approach
#---------------------------------------------------------------------------------------------------------------------------
fnn_params = {'hidden_layer_sizes': [(100,), (100, 50)], 'activation': ['relu', 'tanh']}
# hyperparameter gridsearch 
fnn_clf_bow = GridSearchCV(MLPClassifier(max_iter=1000), fnn_params, cv=5)
fnn_clf_bow.fit(X_train_bow, y_train)

# Predictions and Evaluation
y_pred_fnn_bow = fnn_clf_bow.predict(X_test_tfidf)
print("Best parameters:", fnn_clf_bow.best_params_)
print("\nClassification Report (Feed-forward NN - BoW):")
print(classification_report(y_test, y_pred_fnn_bow))

# Confusion Matrix
conf_matrix_fnn_bow = confusion_matrix(y_test, y_pred_fnn_bow)
sns.heatmap(conf_matrix_fnn_bow, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Feed-forward neural network ~ for tf-idf approach
#---------------------------------------------------------------------------------------------------------------------------
fnn_params = {'hidden_layer_sizes': [(100,), (100, 50)], 'activation': ['relu', 'tanh']}
# hyperparameter gridsearch 
fnn_clf_tfidf = GridSearchCV(MLPClassifier(max_iter=1000), fnn_params, cv=5)
fnn_clf_tfidf.fit(X_train_tfidf, y_train)

# Predictions and Evaluation
y_pred_fnn_tfidf = fnn_clf_tfidf.predict(X_test_tfidf)
print("Best parameters:", fnn_clf_tfidf.best_params_)
print("\nClassification Report (Feed-forward NN - tf-idf):")
print(classification_report(y_test, y_pred_fnn_tfidf))

# Confusion Matrix
conf_matrix_fnn_tfidf = confusion_matrix(y_test, y_pred_fnn_tfidf)
sns.heatmap(conf_matrix_fnn_tfidf, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Feed-forward neural network ~ for embedding approach
#---------------------------------------------------------------------------------------------------------------------------
fnn_params = {'hidden_layer_sizes': [(100,), (100, 50)], 'activation': ['relu', 'tanh']}
# hyperparameter gridsearch 
fnn_clf_emb = GridSearchCV(MLPClassifier(max_iter=1000), fnn_params, cv=5)
fnn_clf_emb.fit(X_train_emb, y_train)

# Predictions and Evaluation
y_pred_fnn_emb = fnn_clf_emb.predict(X_test_emb)
print("Best parameters:", fnn_clf_emb.best_params_)
print("\nClassification Report (Feed-forward NN - embedding):")
print(classification_report(y_test, y_pred_fnn_emb))

# Confusion Matrix
conf_matrix_fnn_emb = confusion_matrix(y_test, y_pred_fnn_emb)
sns.heatmap(conf_matrix_fnn_emb, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Convolutional neural network ~ for BoW approach
#---------------------------------------------------------------------------------------------------------------------------
# prepare features 
y_train_nn = to_categorical(pd.factorize(y_train)[0])
y_val_nn = to_categorical(pd.factorize(y_val)[0])
y_test_nn = to_categorical(pd.factorize(y_test)[0])

# define CNN 
def build_cnn_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=150, output_dim=hp.Int('embedding_output', min_value=32, max_value=128, step=32), input_length=150))
    model.add(Conv1D(filters=hp.Int('filters', min_value=64, max_value=256, step=64), kernel_size=hp.Choice('kernel_size', values=[3, 5, 7]), activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(units=len(y_train_nn[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN tuner
cnn_tuner_bow = RandomSearch(build_cnn_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='cnn_tuner', project_name='CNNHyperparamTuning')
cnn_tuner_bow.search(X_train_bow, y_train_nn, epochs=10, validation_data=(X_val_bow, y_val_nn))

# fit best CNN model 
best_hps_cnn_bow = cnn_tuner_bow.get_best_hyperparameters(num_trials=1)[0]
best_cnn_model_bow = build_cnn_model(best_hps_cnn_bow)
history_cnn_bow = best_cnn_model_bow.fit(X_train_bow, y_train_nn, epochs=10, validation_data=(X_val_bow, y_val_nn))

# predictions and evaluation
y_pred_cnn_bow = best_cnn_model_bow.predict(X_test_bow)
y_pred_cnn_labels_bow = np.argmax(y_pred_cnn_bow, axis=1)
y_test_labels_bow = np.argmax(y_test_nn, axis=1)

print("\nClassification Report (CNN - BoW):")
print(classification_report(y_test_labels_bow, y_pred_cnn_labels_bow))

# confusion matrix
conf_matrix_cnn_bow = confusion_matrix(y_test_labels_bow, y_pred_cnn_labels_bow)
sns.heatmap(conf_matrix_cnn_bow, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# plotting accuracy curves
plt.plot(history_cnn_bow.history['accuracy'])
plt.plot(history_cnn_bow.history['val_accuracy'])
plt.title('CNN Model Accuracy - BoW')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Convolutional neural network ~ for tf-idf approach
#---------------------------------------------------------------------------------------------------------------------------
# prepare features 
y_train_nn = to_categorical(pd.factorize(y_train)[0])
y_val_nn = to_categorical(pd.factorize(y_val)[0])
y_test_nn = to_categorical(pd.factorize(y_test)[0])

# define CNN 
def build_cnn_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=150, output_dim=hp.Int('embedding_output', min_value=32, max_value=128, step=32), input_length=150))
    model.add(Conv1D(filters=hp.Int('filters', min_value=64, max_value=256, step=64), kernel_size=hp.Choice('kernel_size', values=[3, 5, 7]), activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(units=len(y_train_nn[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN tuner
cnn_tuner_tfidf = RandomSearch(build_cnn_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='cnn_tuner', project_name='CNNHyperparamTuning')
cnn_tuner_tfidf.search(X_train_tfidf, y_train_nn, epochs=10, validation_data=(X_val_tfidf, y_val_nn))

# fit best CNN model 
best_hps_cnn_tfidf = cnn_tuner_tfidf.get_best_hyperparameters(num_trials=1)[0]
best_cnn_model_tfidf = build_cnn_model(best_hps_cnn_tfidf)
history_cnn_tfidf = best_cnn_model_tfidf.fit(X_train_tfidf, y_train_nn, epochs=10, validation_data=(X_val_tfidf, y_val_nn))

# predictions and evaluation
y_pred_cnn_tfidf = best_cnn_model_tfidf.predict(X_test_tfidf)
y_pred_cnn_labels_tfidf = np.argmax(y_pred_cnn_tfidf, axis=1)
y_test_labels_tfidf = np.argmax(y_test_nn, axis=1)

print("\nClassification Report (CNN - tf-idf):")
print(classification_report(y_test_labels_tfidf, y_pred_cnn_labels_tfidf))

# confusion matrix
conf_matrix_cnn_tfidf = confusion_matrix(y_test_labels_tfidf, y_pred_cnn_labels_tfidf)
sns.heatmap(conf_matrix_cnn_tfidf, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# plotting accuracy curves
plt.plot(history_cnn_tfidf.history['accuracy'])
plt.plot(history_cnn_tfidf.history['val_accuracy'])
plt.title('CNN Model Accuracy - tf-idf')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Convolutional neural network ~ for embedding approach
#---------------------------------------------------------------------------------------------------------------------------
# prepare features 
y_train_nn = to_categorical(pd.factorize(y_train)[0])
y_val_nn = to_categorical(pd.factorize(y_val)[0])
y_test_nn = to_categorical(pd.factorize(y_test)[0])

# define CNN 
def build_cnn_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=150, output_dim=hp.Int('embedding_output', min_value=32, max_value=128, step=32), input_length=150))
    model.add(Conv1D(filters=hp.Int('filters', min_value=64, max_value=256, step=64), kernel_size=hp.Choice('kernel_size', values=[3, 5, 7]), activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(units=len(y_train_nn[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# CNN tuner
cnn_tuner_emb = RandomSearch(build_cnn_model, objective='val_accuracy', max_trials=10, executions_per_trial=5, directory='cnn_tuner', project_name='CNNHyperparamTuning')
cnn_tuner_emb.search(X_train_emb, y_train_nn, epochs=10, validation_data=(X_val_emb, y_val_nn))

# fit best CNN model 
best_hps_cnn_emb = cnn_tuner_emb.get_best_hyperparameters(num_trials=1)[0]
best_cnn_model_emb = build_cnn_model(best_hps_cnn_emb)
history_cnn_emb = best_cnn_model_emb.fit(X_train_emb, y_train_nn, epochs=10, validation_data=(X_val_emb, y_val_nn))

# predictions and evaluation
y_pred_cnn_emb = best_cnn_model_emb.predict(X_test_emb)
y_pred_cnn_labels_emb = np.argmax(y_pred_cnn_emb, axis=1)
y_test_labels_emb = np.argmax(y_test_nn, axis=1)

print("\nClassification Report (CNN - embedding):")
print(classification_report(y_test_labels_emb, y_pred_cnn_labels_emb))

# confusion matrix
conf_matrix_cnn_emb = confusion_matrix(y_test_labels_emb, y_pred_cnn_labels_emb)
sns.heatmap(conf_matrix_cnn_emb, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# plotting accuracy curves
plt.plot(history_cnn_emb.history['accuracy'])
plt.plot(history_cnn_emb.history['val_accuracy'])
plt.title('CNN Model Accuracy - embedding')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Recurrent neural network ~ for BoW approach
#---------------------------------------------------------------------------------------------------------------------------
# prepare features 
y_train_nn = to_categorical(pd.factorize(y_train)[0])
y_val_nn = to_categorical(pd.factorize(y_val)[0])
y_test_nn = to_categorical(pd.factorize(y_test)[0])

def build_rnn_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=150, output_dim=hp.Int('embedding_output', min_value=32, max_value=128, step=32), input_length=150))
    model.add(SpatialDropout1D(hp.Float('spatial_dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=50, max_value=150, step=25), dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1), recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=len(y_train_nn[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# RNN Tuner
rnn_tuner_bow = RandomSearch(build_rnn_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='rnn_tuner', project_name='RNNHyperparamTuning')
rnn_tuner_bow.search(X_train_bow, y_train_nn, epochs=10, validation_data=(X_val_bow, y_val_nn))

# Fit the best model
best_hps_rnn_bow = rnn_tuner_bow.get_best_hyperparameters(num_trials=1)[0]
best_rnn_model_bow = build_rnn_model(best_hps_rnn_bow)
history_rnn_bow = best_rnn_model_bow.fit(X_train_bow, y_train_nn, epochs=10, validation_data=(X_val_bow, y_val_nn))

# Predictions and Evaluation
y_pred_rnn_bow = best_rnn_model_bow.predict(X_test_bow)
y_pred_rnn_labels_bow = np.argmax(y_pred_rnn_bow, axis=1)
y_test_label_bow = np.argmax(y_test_nn, axis=1)

print("\nClassification Report (RNN - BoW):")
print(classification_report(y_test_label_bow, y_pred_rnn_labels_bow))

# Confusion Matrix
conf_matrix_rnn_bow = confusion_matrix(y_test_label_bow, y_pred_rnn_labels_bow)
sns.heatmap(conf_matrix_rnn_bow, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plotting accuracy curves
plt.plot(history_rnn_bow.history['accuracy'])
plt.plot(history_rnn_bow.history['val_accuracy'])
plt.title('RNN Model Accuracy - BoW')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Recurrent neural network ~ for tf-idf approach
#---------------------------------------------------------------------------------------------------------------------------
# prepare features 
y_train_nn = to_categorical(pd.factorize(y_train)[0])
y_val_nn = to_categorical(pd.factorize(y_val)[0])
y_test_nn = to_categorical(pd.factorize(y_test)[0])

def build_rnn_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=150, output_dim=hp.Int('embedding_output', min_value=32, max_value=128, step=32), input_length=150))
    model.add(SpatialDropout1D(hp.Float('spatial_dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=50, max_value=150, step=25), dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1), recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=len(y_train_nn[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# RNN Tuner
rnn_tuner_tfidf = RandomSearch(build_rnn_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='rnn_tuner', project_name='RNNHyperparamTuning')
rnn_tuner_tfidf.search(X_train_tfidf, y_train_nn, epochs=10, validation_data=(X_val_tfidf, y_val_nn))

# Fit the best model
best_hps_rnn_tfidf = rnn_tuner_tfidf.get_best_hyperparameters(num_trials=1)[0]
best_rnn_model_tfidf = build_rnn_model(best_hps_rnn_tfidf)
history_rnn_tfidf = best_rnn_model_tfidf.fit(X_train_tfidf, y_train_nn, epochs=10, validation_data=(X_val_tfidf, y_val_nn))

# Predictions and Evaluation
y_pred_rnn_tfidf = best_rnn_model_tfidf.predict(X_test_tfidf)
y_pred_rnn_labels_tfidf = np.argmax(y_pred_rnn_tfidf, axis=1)
y_test_label_tfidf = np.argmax(y_test_nn, axis=1)

print("\nClassification Report (RNN - tf-idf):")
print(classification_report(y_test_label_tfidf, y_pred_rnn_labels_tfidf))

# Confusion Matrix
conf_matrix_rnn_tfidf = confusion_matrix(y_test_label_tfidf, y_pred_rnn_labels_tfidf)
sns.heatmap(conf_matrix_rnn_tfidf, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plotting accuracy curves
plt.plot(history_rnn_tfidf.history['accuracy'])
plt.plot(history_rnn_tfidf.history['val_accuracy'])
plt.title('RNN Model Accuracy - tf-idf')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------
# Recurrent neural network ~ for embedding approach
#---------------------------------------------------------------------------------------------------------------------------
# prepare features 
y_train_nn = to_categorical(pd.factorize(y_train)[0])
y_val_nn = to_categorical(pd.factorize(y_val)[0])
y_test_nn = to_categorical(pd.factorize(y_test)[0])

def build_rnn_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=150, output_dim=hp.Int('embedding_output', min_value=32, max_value=128, step=32), input_length=150))
    model.add(SpatialDropout1D(hp.Float('spatial_dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=50, max_value=150, step=25), dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1), recurrent_dropout=hp.Float('recurrent_dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=len(y_train_nn[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# RNN Tuner
rnn_tuner_emb = RandomSearch(build_rnn_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='rnn_tuner', project_name='RNNHyperparamTuning')
rnn_tuner_emb.search(X_train_emb, y_train_nn, epochs=10, validation_data=(X_val_emb, y_val_nn))

# Fit the best model
best_hps_rnn_emb = rnn_tuner_emb.get_best_hyperparameters(num_trials=1)[0]
best_rnn_model_emb = build_rnn_model(best_hps_rnn_emb)
history_rnn_emb = best_rnn_model_emb.fit(X_train_emb, y_train_nn, epochs=10, validation_data=(X_val_emb, y_val_nn))

# Predictions and Evaluation
y_pred_rnn_emb = best_rnn_model_emb.predict(X_test_emb)
y_pred_rnn_labels_emb = np.argmax(y_pred_rnn_emb, axis=1)
y_test_label_emb = np.argmax(y_test_nn, axis=1)

print("\nClassification Report (RNN - embedding):")
print(classification_report(y_test_label_emb, y_pred_rnn_labels_emb))

# Confusion Matrix
conf_matrix_rnn_emb = confusion_matrix(y_test_label_emb, y_pred_rnn_labels_emb)
sns.heatmap(conf_matrix_rnn_emb, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plotting accuracy curves
plt.plot(history_rnn_emb.history['accuracy'])
plt.plot(history_rnn_emb.history['val_accuracy'])
plt.title('RNN Model Accuracy - embedding')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
#---------------------------------------------------------------------------------------------------------------------------